In [ ]:
!pip uninstall -y transformers
!pip install transformers==4.46.3 datasets==3.0.2 accelerate==0.34.2



Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)


In [ ]:
from google.colab import files  #upload dos datasets finais de treino e teste

uploaded = files.upload()


Saving test_dataset.csv to test_dataset (1).csv
Saving train_dataset.csv to train_dataset (1).csv


In [ ]:
import pandas as pd
from datasets import Dataset

train_df = pd.read_csv("/content/train_dataset.csv", sep=";")
test_df = pd.read_csv("/content/test_dataset.csv", sep=";")


train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification   #carregando modelo bert e o tokenizador

MODEL_NAME = "neuralmind/bert-base-portuguese-cased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2  #camada final com duas saidas (binario)
)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["texto"],
        padding="max_length",   # preenche o texto do datasets até o tamanho máximo
        truncation=True,        # corta se for muito longo
        max_length=256          # numero de tokens
    )


train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/11914 [00:00<?, ? examples/s]

Map:   0%|          | 0/2979 [00:00<?, ? examples/s]

In [ ]:
#hugging face para configurar e treinar
from transformers import TrainingArguments, Trainer

# scikit-learn p calcular métricas de desempenho
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np
# função do Trainer para calcular as métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    #calcula metricas
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)


    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:

import os
os.environ["WANDB_DISABLED"] = "true"  # Desativa o weights & biases


training_args = TrainingArguments(
    output_dir="/content/bertimbau_fake_news",
    evaluation_strategy="epoch",               # avalia  a cada época
    save_strategy="epoch",                 # salva checkpoint a cada época
    learning_rate=2e-5,                    # taxa aprendizado
    per_device_train_batch_size=8,         #lote treino
    per_device_eval_batch_size=8,          #lote avaliação
    num_train_epochs=5,
    weight_decay=0.01,             #regula p evitar overfitting
    logging_dir="/content/logs",               #logs
    load_best_model_at_end=True    #carrega melhor modelo no final
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,  # dados tokenizados
    eval_dataset=test_tokenized,    # dados teste tokenizados
    tokenizer=tokenizer,            # tokenizer p texto
    compute_metrics=compute_metrics # função das métricasdesempenho
)


trainer.train()


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-4235962717.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.074700,0.049225,0.989594,0.989594,0.989596,0.989594
2,0.024400,0.058990,0.989930,0.989929,0.989933,0.989930
3,0.011300,0.056609,0.992279,0.992279,0.992281,0.992279
4,0.005500,0.077322,0.988587,0.988587,0.988588,0.988587
5,0.000000,0.068072,0.990265,0.990265,0.990267,0.990265


TrainOutput(global_step=7450, training_loss=0.029666378240682335, metrics={'train_runtime': 3218.4531, 'train_samples_per_second': 18.509, 'train_steps_per_second': 2.315, 'total_flos': 7836762783897600.0, 'train_loss': 0.029666378240682335, 'epoch': 5.0})

In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.04922514036297798, 'eval_accuracy': 0.9895938234306815, 'eval_f1': 0.9895938163950735, 'eval_precision': 0.9895958165550406, 'eval_recall': 0.9895938234306815, 'eval_runtime': 47.8988, 'eval_samples_per_second': 62.194, 'eval_steps_per_second': 7.787, 'epoch': 5.0}


In [ ]:
model.save_pretrained("/content/bertimbau_fakenews_final")
tokenizer.save_pretrained("/content/bertimbau_fakenews_final")


('/content/bertimbau_fakenews_final/tokenizer_config.json',
 '/content/bertimbau_fakenews_final/special_tokens_map.json',
 '/content/bertimbau_fakenews_final/vocab.txt',
 '/content/bertimbau_fakenews_final/added_tokens.json',
 '/content/bertimbau_fakenews_final/tokenizer.json')

In [ ]:
from google.colab import drive   #p salvar no meu drive
drive.mount('/content/drive')

!cp -r /content/bertimbau_fake_news /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/bertimbau_fake_news


checkpoint-1490  checkpoint-4470  checkpoint-7450
checkpoint-2980  checkpoint-5960


In [ ]:
!zip -r /content/modelo_fakenews_final.zip "/content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450"
from google.colab import files
files.download("/content/modelo_fakenews_final.zip")  #zipar o ultimo checkpoint e baixar


  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/ (stored 0%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/config.json (deflated 53%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/tokenizer_config.json (deflated 75%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/special_tokens_map.json (deflated 42%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/vocab.txt (deflated 52%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/tokenizer.json (deflated 72%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/training_args.bin (deflated 53%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/optimizer.pt (deflated 15%)
  adding: content/drive/MyDrive/bertimbau_fake_news/checkpoint-7450/scheduler.pt (deflated 61%)
  adding: content/drive/MyDr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>